In [ ]:
from jobs import Sparkinit
from json import dump, dumps
from utils import formatar_sql
spark_start = Sparkinit()
import os
spark = spark_start.buscar_sessao_spark()

print(f"WebUI SparkJobs: {spark.sparkContext.uiWebUrl}")
spark.getActiveSession()

# Vendas

In [ ]:
import os
# Carregando o dataframe para verificar a estrutura das colunas e tipos de dados
path = os.path.abspath(r"C:\Users\gustavo.lopes\Documentos\GitHub\metodologia_medalhao_pipeline_engenharia_de_dados\datalake\camada_0_transient\VENDAS")
dados = spark.read.format("parquet").load(path)

dados.printSchema()

dados.show(n=1, vertical=True)
colunas = ',\n\t'.join(dados.columns)

base = [f'CAST({col} AS STRING)' for col in dados.columns]
base_media = ' , '.join(base)
base_final = f"BASE64(CONCAT({base_media})) AS hash_id"
base_final

f"""/* 'Bronze' */
SELECT 
        {colunas},
        {base_final}
FROM vendas_tmp
"""


In [ ]:
# Criando a tabela temporária e realizando a consulta que depois usaremos para transformar a camada raw
dados.createOrReplaceTempView("vendas_tmp")

query = formatar_sql("""/* 'Silver' */
SELECT 
    COALESCE(date_format(vt.d_dt_vd, 'yyyy-MM-dd HH:mm:ss'), '') AS data_emissao,
    CAST(vt.n_id_fil AS BIGINT) AS codigo_filial,
    CAST(vt.n_id_vd_fil AS BIGINT) AS id_venda_filial,
    COALESCE(CAST(vt.v_cli_cod AS STRING), '') AS codigo_cliente,
    CAST(vt.n_vlr_tot_vd AS DECIMAL(38, 2)) AS valor_total_venda,
    CAST(vt.n_vlr_tot_desc AS DECIMAL(38, 2)) AS valor_total_desconto,
    CASE 
        WHEN vt.v_cpn_eml  = 'SIM' THEN True
        ELSE False
    END AS enviado_email,
    COALESCE(CAST(vt.tp_pgt AS STRING), '') AS tipo_pagamento
    
FROM vendas_tmp as vt""")

spark.sql(query).show()
query

# Pedidos

In [ ]:

# Carregando o dataframe para verificar a estrutura das colunas e tipos de dados
dados = spark.read.format("parquet").load(os.path.abspath(r"C:\Users\gustavo.lopes\Documentos\GitHub\metodologia_medalhao_pipeline_engenharia_de_dados\datalake\camada_0_transient\PEDIDOS"))

dados.printSchema()

dados.show(n=1, vertical=True)
colunas = ',\n\t'.join(dados.columns)
base = [f'CAST({col} AS STRING)' for col in dados.columns]
base_media = ' , '.join(base)
base_final = f"BASE64(CONCAT({base_media})) AS hash_id"
base_final

f"""/* 'Bronze' */
SELECT 
        {colunas}, 
        {base_final}
FROM pedidos_tmp
"""

In [ ]:
# Criando a tabela temporária e realizando a consulta que depois usaremos para transformar a camada raw
dados.createOrReplaceTempView("pedidos_tmp")

query = formatar_sql("""/* 'Silver' */
SELECT 
    CAST(pd.n_id_pdd AS BIGINT) AS id_pedido,
    COALESCE(CAST(pd.d_dt_eft_pdd AS DATE), '') AS data_realizacao_pedido,
    COALESCE(date_format(pd.d_dt_entr_pdd, 'yyyy-MM-dd HH:mm:ss'), '') AS data_entrega,
    CASE 
        WHEN pd.v_cnl_orig_pdd = 'L' THEN 'Loja'
        WHEN pd.v_cnl_orig_pdd = 'A' THEN 'App'
        WHEN pd.v_cnl_orig_pdd = 'S' THEN 'Site'
    END AS canal_venda,
    COALESCE(CAST(pd.v_uf_entr_pdd AS STRING), '') AS UF_pedido,
    COALESCE(CAST(pd.v_lc_ent_pdd AS STRING), '') AS cidade_entrega,
    CAST(pd.n_vlr_tot_pdd AS DECIMAL(38,2)) AS valor_total_pedido
    BASE64(CAST(n_id_pdd AS STRING)) AS hash_id
FROM pedidos_tmp as pd""")

spark.sql(query).show()
query

# Pedidos Vendas

In [ ]:

# Carregando o dataframe para verificar a estrutura das colunas e tipos de dados
dados = spark.read.format("parquet").load(os.path.abspath(r"C:\Users\gustavo.lopes\Documentos\GitHub\metodologia_medalhao_pipeline_engenharia_de_dados\datalake\camada_0_transient\PEDIDO_VENDA"))

dados.printSchema()

dados.show(n=1, vertical=True)
colunas = ',\n\t'.join(dados.columns)
base = [f'CAST({col} AS STRING)' for col in dados.columns]
base_media = ' , '.join(base)
base_final = f"BASE64(CONCAT({base_media})) AS hash_id"
base_final
f"""/* 'Bronze' */
SELECT 
        {colunas},
        {base_final} 
FROM pedido_venda_tmp
"""

In [ ]:
# Criando a tabela temporária e realizando a consulta que depois usaremos para transformar a camada raw
dados.createOrReplaceTempView("pedido_venda_tmp")

query = formatar_sql("""/* 'Silver'*/
SELECT 
    CAST(pv.n_id_fil AS BIGINT) AS codigo_filial,
    CAST(pv.n_id_vd_fil AS BIGINT) AS id_venda_filial,
    CAST(pv.n_id_pdd AS BIGINT) AS id_pedido
    BASE64(CONCAT(CAST(n_id_fil AS STRIN), CAST(n_id_vd_fil AS STRING), CAST(n_id_pdd AS STRING))) AS hash_id
FROM pedido_venda_tmp as pv""")



spark.sql(query).show()
query

# Itens Vendas

In [ ]:

# Carregando o dataframe para verificar a estrutura das colunas e tipos de dados
dados = spark.read.format("parquet").load(os.path.abspath(r"C:\Users\gustavo.lopes\Documentos\GitHub\metodologia_medalhao_pipeline_engenharia_de_dados\datalake\camada_0_transient\ITENS_VENDAS"))

dados.printSchema()

dados.show(n=1, vertical=True)
colunas = ',\n\t'.join(dados.columns)
base = [f'CAST({col} AS STRING)' for col in dados.columns]
base_media = ' , '.join(base)
base_final = f"BASE64(CONCAT({base_media})) AS hash_id"
base_final
f"""/* 'Bronze' */
SELECT 
        {colunas},
        {base_final}
FROM itens_vendas_tmp
"""


In [ ]:
# Criando a tabela temporária e realizando a consulta que depois usaremos para transformar a camada raw
dados.createOrReplaceTempView("itens_vendas_tmp")

query = formatar_sql("""/* 'Silver' */
SELECT 
    CAST(iv.n_id_fil AS BIGINT) AS codigo_filial,
    CAST(iv.n_id_vd_fil AS BIGINT) AS id_venda_filial,
    CAST(iv.n_id_it AS BIGINT) AS codigo_item_venda,
    CASE 
        WHEN iv.v_rc_elt  = 'SIM' THEN True
        ELSE False
    END AS com_receita_eletronica,
    CASE 
        WHEN iv.v_it_vd_conv = 'SIM' THEN 'Convênio'
        WHEN iv.v_it_vd_conv = 'NAO' AND iv.n_vlr_desc > 0 THEN 'Promoção'
        ELSE 'Sem Desconto'
    END AS tipo_desconto,
    CAST(iv.n_vlr_pis AS DECIMAL(38,2)) AS valor_pis_item,
    CAST(iv.n_vlr_vd AS DECIMAL(38,2)) AS valor_final_item,
    CAST(iv.n_vlr_desc AS DECIMAL(38,2)) AS valor_desconto_item,
    CAST(iv.n_qtd AS BIGINT) AS quantidade_itens,
FROM itens_vendas_tmp as iv""")

spark.sql(query).show()
query

# Endereços Clientes

In [ ]:

# Carregando o dataframe para verificar a estrutura das colunas e tipos de dados
dados = spark.read.format("parquet").load(os.path.abspath(r"C:\Users\gustavo.lopes\Documentos\GitHub\metodologia_medalhao_pipeline_engenharia_de_dados\datalake\camada_0_transient\ENDERECOS_CLIENTES"))

dados.printSchema()

dados.show(n=1, vertical=True)
colunas = ',\n\t'.join(dados.columns)
base = [f'CAST({col} AS STRING)' for col in dados.columns]
base_media = ' , '.join(base)
base_final = f"BASE64(CONCAT({base_media})) AS hash_id"
base_final
f"""/* 'Bronze' */
SELECT 
        {colunas},
        {base_final}
FROM enderecos_clientes_tmp
"""

In [ ]:
# Criando a tabela temporária e realizando a consulta que depois usaremos para transformar a camada raw
dados.createOrReplaceTempView("enderecos_clientes_tmp")

query = formatar_sql("""/* 'Silver' */
SELECT 
    CAST(ec.v_id_cli AS STRING) AS codigo_cliente,
    CAST(ec.n_sq_end AS BIGINT) AS sequencia_endereco_cliente,
    COALESCE(date_format(ec.d_dt_exc, 'yyyy-MM-dd HH:mm:ss'), '') AS data_exclusao_endereco,
    CAST(ec.v_lcl AS STRING) AS cidade_endereco,
    CAST(ec.v_uf AS STRING) AS UF_endereco
FROM enderecos_clientes_tmp as ec""")

spark.sql(query).show()
query

# Clientes Opt

In [ ]:
# Carregando o dataframe para verificar a estrutura das colunas e tipos de dados
dados = spark.read.option("multiline","true").format("json").load(os.path.abspath(r"C:\Users\gustavo.lopes\Documentos\GitHub\metodologia_medalhao_pipeline_engenharia_de_dados\datalake\camada_0_transient\CLIENTES_OPT"))

dados.printSchema()

dados.show(n=2, vertical=True)
colunas = ',\n\t'.join(dados.columns)
base = ['CAST({} AS STRING)'.format(col) for col in dados.columns]
base_media = ' , '.join(base)
base_final = f"BASE64(CONCAT({base_media})) AS hash_id"
base_final
f"""/* 'Bronze' */
SELECT 
        {colunas},
        {base_final}
FROM clientes_opt_tmp"""


In [ ]:
# Criando a tabela temporária e realizando a consulta que depois usaremos para transformar a camada raw
dados.createOrReplaceTempView("clientes_opt_tmp")

query = formatar_sql("""/* 'Silver' */
SELECT 
    CAST(co.v_id_cli AS STRING) AS codigo_cliente,
    CASE 
        WHEN co.b_push  = True THEN 'SIM'
        WHEN co.b_push  = False THEN 'NÃO'
        ELSE ''
    END AS autoriza_notificacao_push,
    CASE 
        WHEN co.b_sms  = True THEN 'SIM'
        WHEN co.b_sms  = False THEN 'NÃO'
        ELSE ''
    END AS autoriza_notificacao_sms,
    CASE 
        WHEN co.b_email  = True THEN 'SIM'
        WHEN co.b_email  = False THEN 'NÃO'
        ELSE ''
    END AS autoriza_notificacao_email,
    CASE 
        WHEN co.b_call  = True THEN 'SIM'
        WHEN co.b_call  = False THEN 'NÃO'
        ELSE ''
    END AS autoriza_notificacao_ligacao
FROM clientes_opt_tmp as co""")

spark.sql(query).show()
query

# Clientes

In [ ]:
# Carregando o dataframe para verificar a estrutura das colunas e tipos de dados
dados = spark.read.format("parquet").load(os.path.abspath(os.path.join(os.getcwd(), "datalake/camada_0_transient/CLIENTES")))

dados.printSchema()

dados.show(n=1, vertical=True)
colunas = ',\n\t'.join(dados.columns)
base = ['CAST({} AS STRING)'.format(col) for col in dados.columns]
base_media = ' , '.join(base)
base_final = f"BASE64(CONCAT({base_media})) AS hash_id"
base_final
print(dumps(f"""/* 'Bronze' */
SELECT 
        {colunas},
        {base_final}
FROM clientes_tmp"""))


In [ ]:
# Criando a tabela temporária e realizando a consulta que depois usaremos para transformar a camada raw
dados.createOrReplaceTempView("clientes_tmp")

query = formatar_sql("""/* 'Silver' */
SELECT 
    CAST(c.v_id_cli AS STRING) AS codigo_cliente,
    COALESCE(CAST(c.d_dt_nasc AS DATE), '') AS data_nascimento_cliente,
    CAST(DATEDIFF(CURRENT_DATE(), CAST(c.d_dt_nasc AS DATE)) / 365 AS INT) as idade,
    CASE 
        WHEN c.v_sx_cli  = 'F' THEN 'Feminino'
        WHEN c.v_sx_cli  = 'M' THEN 'Masculino'
        ELSE '' 
    END AS genero_biologico_cliente,
    CASE 
        WHEN c.n_est_cvl  = 1 THEN 'Solteiro'
        WHEN c.n_est_cvl  = 2 THEN 'Casado'
        WHEN c.n_est_cvl  = 3 THEN 'Viúvo'
        WHEN c.n_est_cvl  = 4 THEN 'Desquitado'
        WHEN c.n_est_cvl  = 5 THEN 'Divorciado'
        WHEN c.n_est_cvl  = 6 THEN 'Outros'
        ELSE ''
    END AS estado_civil_cliente
FROM clientes_tmp as c""")
spark.sql(query).show()
query

In [ ]:
""" -- 'Tabela Endereços Clientes parquet'
SELECT 
    CAST(c.v_id_cli AS STRING) AS codigo_cliente,
    COALESCE(CAST(c.d_dt_nasc AS DATE), '') AS data_nascimento_cliente,
    CASE 
        WHEN c.v_sx_cli  = 'F' THEN 'Feminino'
        WHEN c.v_sx_cli  = 'M' THEN 'Masculino'
        ELSE '' 
    END AS genero_biologico_cliente,
    CASE 
        WHEN c.n_est_cvl  = 1 THEN 'Solteiro'
        WHEN c.n_est_cvl  = 2 THEN 'Casado'
        WHEN c.n_est_cvl  = 3 THEN 'Viúvo'
        WHEN c.n_est_cvl  = 4 THEN 'Desquitado'
        WHEN c.n_est_cvl  = 5 THEN 'Divorciado'
        WHEN c.n_est_cvl  = 6 THEN 'Outros'
        ELSE ''
    END AS estado_civil_cliente
FROM clientes_tmp as c"""

In [ ]:
""" /* Gold 'Vendas' */
SELECT 
    v.codigo_filial AS codifo_filial,
    v.id_venda_filial AS codigo_cupom_venda,
    v.data_emissao AS data_emissao,
    iv.codigo_item_venda AS codigo_item,
    iv.valor_final_item AS valor_unitario,
    iv.quantidade_itens AS quantidade,
    v.codigo_cliente AS codigo_cliente,
    iv.tipo_desconto AS tipo_desconto,
    p.canal_venda AS canal_venda
FROM vendas AS v
INNER JOIN pedido_venda AS pv ON CONCAT(CAST(pv.codigo_filial AS STRING), CAST(pv.id_venda_filial AS STRING)) = CONCAT(CAST(v.codigo_filial AS STRING), CAST(v.id_venda_filial AS STRING))
INNER JOIN itens_vendas AS iv ON CONCAT(CAST(iv.codigo_filial AS STRING), CAST(iv.id_venda_filial)) = CONCAT(CAST(v.codigo_filial AS STRING), CAST(v.id_venda_filial AS STRING))
INNER JOIN pedidos AS p ON p.id_pedido = pv.id_pedido
"""

In [ ]:
print("/* 'Silver' */\nSELECT COALESCE(date_format(vt.d_dt_vd, 'yyyy-MM-dd HH:mm:ss'), '') AS data_emissao,\n       CAST(vt.n_id_fil AS BIGINT) AS codigo_filial,\n       CAST(vt.n_id_vd_fil AS BIGINT) AS id_venda_filial,\n       COALESCE(CAST(vt.v_cli_cod AS STRING), '') AS codigo_cliente,\n       CAST(vt.n_vlr_tot_vd AS DECIMAL(38, 2)) AS valor_total_venda,\n       CAST(vt.n_vlr_tot_desc AS DECIMAL(38, 2)) AS valor_total_desconto,\n       CASE\n           WHEN vt.v_cpn_eml = 'SIM' THEN TRUE\n           ELSE FALSE\n       END AS enviado_email,\n       COALESCE(CAST(vt.tp_pgt AS STRING), '') AS tipo_pagamento, \nhash_id\nFROM vendas_tmp AS vt")

In [ ]:
"""/* Gold 'Clientes' */
SELECT
    c.codigo_cliente AS codigo_cliente,
    c.data_nascimento_cliente AS data_nascimento,
    DATEDIFF('YEAR', CAST(data_nascimento AS DATE), CURRENT_DATE()) as idade,
    c.genero_biologico_cliente AS sexo,
    ec.UF_endereco AS uf,
    ec.cidade_endereco AS cidade,
    c.estado_civil_cliente AS estado_civil,
    co.autoriza_notificacao_ligacao AS flag_lgpd_call,
    co.autoriza_notificacao_sms AS flag_lgpd_sms,
    co.autoriza_notificacao_email AS flag_lgpd_email,
    co.autoriza_notificacao_push AS flag_lgpd_push
FROM clientes AS c
INNER JOIN clientes_opt AS co ON co.codigo_cliente = c.codigo_cliente
INNER JOIN enderecos_clientes AS ec ON ec.codigo_cliente = c.codigo_cliente"""